In [10]:
import ee
import json
import requests
import geopandas as gpd

In [4]:
# Authenticate
#ee.Authenticate()

# Initialize API
ee.Initialize()

In [17]:
with open('/home/alejandro/Documentos/GIS/El_Pilar/Area_Interes.geojson') as file:
    aoi_json = json.load(file)
aoi = ee.Geometry(aoi_json['features'][0]['geometry'])
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2024-06-01', '2024-08-09').filterBounds(aoi)

In [43]:
for image in sentinel2.getInfo()['features']:
    # Obtiene el id y la fecha de la imagen
    date = ee.Date(image['properties']['system:time_start']).format('YYYY-MM-dd').getInfo()

    # Crea el nombre del archivo
    filename = f"{date}.tif"

    # Selecciona la imagen y recorta con el área de interés
    img = ee.Image(image['id']).select(['B4', 'B3', 'B2']).clip(aoi)

    # Descarga la imagen
    url = img.getDownloadUrl({
        'name': filename,
        'scale': 10,  # escala en metros
        'crs': 'EPSG:4326',  # sistema de coordenadas
        'format': 'GEO_TIFF'
    })

    # Descarga el archivo
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)